In [ ]:
import sys
sys.path.append('..')
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from plotting import tsplot, tsplot_boot,show_landmark_and_platform
from tqdm import tqdm_notebook as tqdm
import random

In [ ]:
from environments import WaterMazeEnv
from combined_agent import Agent

import os 
output_path = 'figures/'

In [ ]:
envi = WaterMazeEnv()

In [ ]:
ag = Agent(env=envi,lesion_hippocampus=True)

In [ ]:
n_simulations = 30
n_episodes = 15


escape_times = np.zeros((n_simulations, n_episodes))

for sim in tqdm(range(n_simulations)):
    ag = Agent(env=envi,lesion_hippocampus=False)
    for ep in range(n_episodes):
        t, reward, locs, choices = ag.train_one_episode()
        escape_times[sim, ep] = t

In [ ]:
fig, ax = plt.subplots()
tsplot_boot(ax,escape_times)

In [ ]:
plt.show()

In [ ]:
ag.env.landmark_1_centre

In [ ]:
ag.env.platform_centre

In [ ]:
def get_platform_and_landmark_locations(env, number=9):
    angles = np.linspace(0, 2*np.pi, number)
    r = env.maze_radius / 2
    platform_locations = [[r * np.cos(a), r * np.sin(a)] for a in angles]
    landmark_locations = [[r * np.cos(a), r * np.sin(a)+.1] for a in angles]
    platform_locations = env.maze_centre + platform_locations
    landmark_locations = env.maze_centre + landmark_locations
    return platform_locations, landmark_locations

In [ ]:
platform_locations, landmark_locations = get_platform_and_landmark_locations(envi)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(np.array(platform_locations)[:,0], np.array(platform_locations)[:,1])
plt.scatter(np.array(landmark_locations)[:,0], np.array(landmark_locations)[:,1])

plt.show()

In [ ]:
platform_locations

In [ ]:
n_sims = 50
n_trials = 4
n_sessions = 12

escape_times = np.zeros((n_sims, n_sessions, n_trials))
session_ids = list(range(len(platform_locations)))
sessions = [np.random.choice(session_ids) for _ in range(n_sessions)]

sessions = [0, 4, 1, 5, 2, 6, 3, 7, 4, 8, 0, 2]

first_trials_control=[]

sim=0
for sim in tqdm(range(n_sims)):
    ag = Agent(env=envi)
    ag.hippocampus.max_goal_response = 20
    ag.striatum.lamb= .65
    ag.epsilon=.4
    for sess_n, session in enumerate(sessions):
        envi.platform_centre = platform_locations[session]
        envi.landmark_1_centre = landmark_locations[session]

        for trial in range(n_trials):
            t, reward, locs, choices = ag.train_one_episode()
            if trial == 0:
                first_trials_control.append(locs)
            escape_times[sim, sess_n, trial] = t


In [ ]:
escape_times_lesion = np.zeros((n_sims, n_sessions, n_trials))
sim=0

first_trials_lesion=[]

for sim in tqdm(range(n_sims)):
    ag = Agent(env=envi, lesion_hippocampus=True)
    ag.striatum.lamb= .76
    ag.striatum.learning_rate = .0008
    for sess_n, session in enumerate(sessions):
        envi.platform_centre = platform_locations[session]
        envi.landmark_1_centre = landmark_locations[session]

        for trial in range(n_trials):
            t, reward, locs, choices = ag.train_one_episode()
            if trial == 0:
                first_trials_lesion.append(locs)
            escape_times_lesion[sim, sess_n, trial] = t
    


In [ ]:
# plot the pearce et al study first for the healthy agents
time_step = .13

first_trials = escape_times[:,:,0].mean(axis=0) * time_step
fourth_trials = escape_times[:,:,1].mean(axis=0)* time_step

first_trials_lesion = escape_times_lesion[:,:,0].mean(axis=0) * time_step
fourth_trials_lesion = escape_times_lesion[:,:,1].mean(axis=0) * time_step


In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
red = '#f9868a'
blue = '#86c3f9'



plt.plot(np.arange(1,12), first_trials_lesion[:-1], 'o-', color=red)
plt.plot(np.arange(1,12), first_trials[:-1], 'o-', fillstyle='none', color=blue)
plt.plot(np.arange(1,12), fourth_trials_lesion[:-1], 'o--', color=red)
plt.plot(np.arange(1,12), fourth_trials[:-1], 'o--', fillstyle='none', color=blue)

# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.ylim([0,122])
plt.ylabel('Escape latency (s)', fontsize=15)
plt.xlabel('Session', fontsize=15)

plt.legend(['Hippocampal lesion - trial 1', 'Control - trial 1', 
            'Hippocampal lesion - trial 4', 'Control - trial 4'], fontsize=12)

plt.xticks(np.arange(0, 12))

plt.savefig(os.path.join(output_path,'PearceModel-ClearLegend.svg'))


In [ ]:
%matplotlib notebook

In [ ]:
# How do I get slow but true learning? Run with lambda 76 or higher tomorrow

In [ ]:
def show_maze(ax, env, **kwargs):
    angles = np.linspace(0, 2 * np.pi, 100)
    x_marks = np.cos(angles) * env.maze_radius + env.maze_centre[0]
    y_marks = np.sin(angles) * env.maze_radius + env.maze_centre[1]
    ax.plot(x_marks, y_marks, color='k', LineWidth=3)

In [ ]:
def plot_trajectories(ax, session_nr, all_paths, **kwargs):
    plt.sca(ax)
    ax.plot(all_paths[session_nr][:,0], all_paths[session_nr][:,1], **kwargs)
    platform_loc = platform_locations[sessions[session_nr]]
    previous_platform_loc =  platform_locations[sessions[session_nr-1]]

    platform = plt.Circle(platform_loc, envi.platform_radius, color='k', fill=None)
    ax.add_artist(platform)
    platform_previous = plt.Circle(previous_platform_loc, envi.platform_radius, linestyle='solid', fill=True,
                                  color='gray')
    ax.add_artist(platform_previous)

    show_maze(ax, envi, **kwargs)

    plt.box('off')
    plt.axis('equal')
    plt.xticks([])
    plt.yticks([])
    

In [ ]:
fig, axs = plt.subplots(2,2)
plot_trajectories(axs[0,0], 7, first_trials_control, color=blue)
plot_trajectories(axs[0,1], 7, first_trials_lesion, color=red)
plot_trajectories(axs[1,0], 8, first_trials_control, color=blue)
plot_trajectories(axs[1,1], 8, first_trials_lesion, color=red)
plt.tight_layout(w_pad=-4, h_pad=4)

plt.savefig(os.path.join(output_path,'PearceTrajectories.svg'))

